In [1]:
#Importamos librerías

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request
import requests
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
colonia = []
precios = []
cuartos = []
banos = []
area = []


for i in range(0,51):
    my_url = "https://www.inmuebles24.com/inmuebles-en-venta-en-distrito-federal-pagina-{}.html".format(i)
    
    req = Request(my_url, headers={'User-Agent': 'Mozilla/5.0'})

    uClient = uReq(req)
    page_html = uClient.read()
    uClient.close()

    page_soup2 = soup(page_html, "html.parser")
    
    
    precio = page_soup2.findAll("span", {"class":"firstPrice"})
    direccion = page_soup2.findAll("span", {"class":"postingCardLocation go-to-posting"})
    features = page_soup2.findAll("ul", {"class":"postingCardMainFeatures go-to-posting"})
    
    for i in range(len(precio)):
        try:
            precios.append(precio[i].text)
        except:
            precios.append(None)

    for i in range(len(direccion)):
        try:
            colonia.append(direccion[i].text)
        except:
            colonia.append(None)
    
    for each in features:
        try:
            cuartos.append((each.find("i", {"class":"postingCardIconsFeatures iconBedrooms"}).next_sibling))
        except:
            cuartos.append(None)  
    
    for each in features:
        try:
            banos.append((each.find("i", {"class":"postingCardIconsFeatures iconBathrooms"}).next_sibling))
        except:
            banos.append(None)  
      
    for each in features:
        try:
            area.append((each.find("i", {"class":"postingCardIconsFeatures iconArea"}).next_sibling))
        except:
            area.append(None)
            

# Dataframe

In [4]:
names = ["precio", "cuartos", "baños", "area", "colonia"]

df = pd.DataFrame(columns=names)
df["precio"] = precios
df["cuartos"] = cuartos
df["baños"] = banos
df["area"] = area
df["colonia"] = colonia

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df.head()

,precio,cuartos,baños,area,colonia
0,"\n\t\t\t\t\t\tMN 10,800,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t2 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t2\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t136 m²\n\t\t\t\t,"\n Polanco, Miguel Hidalgo\n"
1,"\n\t\t\t\t\t\tMN 19,728,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t2 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t2\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t211 m²\n\t\t\t\t,"\n Polanco, Miguel Hidalgo\n"
2,"\n\t\t\t\t\t\tMN 10,950,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t2\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t229 m²\n\t\t\t\t,"\n Lomas de Bezares, Miguel Hidalgo\n"
3,"\n\t\t\t\t\t\tMN 13,900,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t316 m²\n\t\t\t\t,"\n Bosque de las Lomas, Miguel Hidalgo\n"
4,"\n\t\t\t\t\t\tUSD 800,000\n\t\t\t\t\t",\n\t\t\t\t\t\t\t\t3 rec.\n\t\t\t\t\t\t\t,\n\t\t\t\t\t3\n\t\t\t\t\t\n\t\t\t\t\t\tbaños\n...,\n\t\t\t\t\t350 m²\n\t\t\t\t,"\n Bosque de las Lomas, Miguel Hidalgo\n"


In [6]:
#Limpiamos dataframe

df["precio"] = df["precio"].str.replace("\n","").str.replace(",","").str.replace("\t","").str.replace("MN","").str.replace("USD","").astype(float)

df["colonia"] = df["colonia"].str.replace("\n","")

df["cuartos"] = df["cuartos"].str.extract("(\d+)").astype(int)

df["baños"] = df["baños"].str.extract("(\d+)").astype(int)

df["area"] = df["area"].str.extract("(\d+)").astype(int)

In [7]:
df.head()

,precio,cuartos,baños,area,colonia
0,10800000.0,2,2,136,"Polanco, Miguel Hidalgo"
1,19728000.0,2,2,211,"Polanco, Miguel Hidalgo"
2,10950000.0,3,2,229,"Lomas de Bezares, Miguel Hidalgo"
3,13900000.0,3,3,316,"Bosque de las Lomas, Miguel Hidalgo"
4,800000.0,3,3,350,"Bosque de las Lomas, Miguel Hidalgo"


In [8]:
#Dividimos colonia y delegacion

new = df["colonia"].str.rsplit(",", n=1, expand=True)

df["delegacion"] = new[1]
df["colonia"] = new[0]

df.drop_duplicates(inplace=True)


In [9]:
df.head()

,precio,cuartos,baños,area,colonia,delegacion
0,10800000.0,2,2,136,Polanco,Miguel Hidalgo
1,19728000.0,2,2,211,Polanco,Miguel Hidalgo
2,10950000.0,3,2,229,Lomas de Bezares,Miguel Hidalgo
3,13900000.0,3,3,316,Bosque de las Lomas,Miguel Hidalgo
4,800000.0,3,3,350,Bosque de las Lomas,Miguel Hidalgo


In [10]:
#Guardamos en formato CSV

df.to_csv(r"C:\Users\Corleone\Anaconda\Inmuebles\Inmuebles24.csv", index=False, encoding='utf-8-sig')